![DLI Header](images/DLI_Header.png)

# 随机人物
｜Random Personas

在此笔记本中，您将学习如何控制模型生成的**温度** ，从而使您能够控制其响应的随机程度。利用**温度** ，您将能够创建各种 AI 个性。

## 学习目标

在完成此笔记本时，您将能够：
- 解释随机**采样（sampling）** 如何导致非确定性 LLM 响应。
- 通过调整**温度（temperature）** 来控制响应的随机程度。

## 视频演练

执行下面的单元格以加载此笔记本的视频演练。

In [ ]:
from IPython.display import HTML

video_url = "https://d36m44n9vdbmda.cloudfront.net/assets/s-fx-12-v1/v2/05-random.mp4"

video_html = f"""
<video controls width="640" height="360">
    <source src="{video_url}" type="video/mp4">
    Your browser does not support the video tag.
</video>
"""

display(HTML(video_html))

## 创建 LLaMA-2 管道
｜Create LLaMA-2 Pipeline

In [ ]:
from transformers import pipeline
model = "TheBloke/Llama-2-13B-chat-GPTQ"
# model = "TheBloke/Llama-2-7B-chat-GPTQ"

llama_pipe = pipeline("text-generation", model=model, device_map="auto");

## 辅助函数

在此笔记本中，我们将使用以下函数来支持我们与 LLM 的交互。现在可以随意浏览一下，因为在下面使用时会更详细地介绍。

### 生成模型响应
｜Generate Model Responses

In [ ]:
def generate(prompt, max_length=1024, pipe=llama_pipe, **kwargs):
    """
    Generates a response to the given prompt using a specified language model pipeline.

    This function takes a prompt and passes it to a language model pipeline, such as LLaMA, 
    to generate a text response. The function is designed to allow customization of the 
    generation process through various parameters and keyword arguments.

    Parameters:
    - prompt (str): The input text prompt to generate a response for.
    - max_length (int): The maximum length of the generated response. Default is 1024 tokens.
    - pipe (callable): The language model pipeline function used for generation. Default is llama_pipe.
    - **kwargs: Additional keyword arguments that are passed to the pipeline function.

    Returns:
    - str: The generated text response from the model, trimmed of leading and trailing whitespace.

    Example usage:
    ```
    prompt_text = "Explain the theory of relativity."
    response = generate(prompt_text, max_length=512, pipe=my_custom_pipeline, temperature=0.7)
    print(response)
    ```
    """

    def_kwargs = dict(return_full_text=False, return_dict=False)
    response = pipe(prompt.strip(), max_length=max_length, **kwargs, **def_kwargs)
    return response[0]['generated_text'].strip()

### 构造提示，可选地使用系统上下文和/或示例
｜Costruct Prompt, Optionally With System Context and/or Examples

In [ ]:
def construct_prompt_with_context(main_prompt, system_context="", conversation_examples=[]):
    """
    Constructs a complete structured prompt for a language model, including optional system context and conversation examples.

    This function compiles a prompt that can be directly used for generating responses from a language model. 
    It creates a structured format that begins with an optional system context message, appends a series of conversational 
    examples as prior interactions, and ends with the main user prompt. If no system context or conversation examples are provided,
    it will return only the main prompt.

    Parameters:
    - main_prompt (str): The core question or statement for the language model to respond to.
    - system_context (str, optional): Additional context or information about the scenario or environment. Defaults to an empty string.
    - conversation_examples (list of tuples, optional): Prior exchanges provided as context, where each tuple contains a user message 
      and a corresponding agent response. Defaults to an empty list.

    Returns:
    - str: A string formatted as a complete prompt ready for language model input. If no system context or examples are provided, returns the main prompt.

    Example usage:
    ```
    main_prompt = "I'm looking to improve my dialogue writing skills for my next short story. Any suggestions?"
    system_context = "User is an aspiring author seeking to enhance dialogue writing techniques."
    conversation_examples = [
        ("How can dialogue contribute to character development?", "Dialogue should reveal character traits and show personal growth over the story arc."),
        ("What are some common pitfalls in writing dialogue?", "Avoid exposition dumps in dialogue and make sure each character's voice is distinct.")
    ]

    full_prompt = construct_prompt_with_context(main_prompt, system_context, conversation_examples)
    print(full_prompt)
    ```
    """
    
    # Return the main prompt if no system context or conversation examples are provided
    if not system_context and not conversation_examples:
        return main_prompt

    # Start with the initial part of the prompt including the system context, if provided
    full_prompt = f"<s>[INST] <<SYS>>{system_context}<</SYS>>\n" if system_context else "<s>[INST]\n"

    # Add each example from the conversation_examples to the prompt
    for user_msg, agent_response in conversation_examples:
        full_prompt += f"{user_msg} [/INST] {agent_response} </s><s>[INST]"

    # Add the main user prompt at the end
    full_prompt += f"{main_prompt} [/INST]"

    return full_prompt

## 默认非随机响应
｜Default Non-random Responses

您可能已经注意到，在我们之前的笔记本中，我们从 LLaMA-2 模型获得的响应，假设我们没有编辑提示，是确定性的。让我们明确地看一下这一点。

在这里，我们将使用我们的模型执行另一个生成任务，这次是生成他们 Galaxy Rider 山地自行车的虚构客户体验。在这里，我们将**系统上下文** 设置为适当的值，并提示模型回忆起他们骑自行车时难忘的一天。

In [ ]:
system_context = """
You are a perennially satisfied customer who loves to reminisce about personal experiences with products. \
You never delve into technical specifics, as you believe it's the emotion and the joy that matter most. \
You're excited for others to feel the same euphoria and happiness you do. Your aim isn't to advertise, \
but to share a genuine, heartfelt story of joy and contentment.
"""

prompt = "Recall a memorable day out with your Galaxy Rider mountain bike in 50 words or less."

In [ ]:
print(generate(construct_prompt_with_context(prompt, system_context)))

---

让我们使用完全相同的提示再生成几次响应。

In [ ]:
print(generate(construct_prompt_with_context(prompt, system_context)))

In [ ]:
print(generate(construct_prompt_with_context(prompt, system_context)))

---

如上所述，模型每次都生成完全相同的响应。对于许多场景，这正是我们想要的行为，但对于其他场景，我们希望在模型的响应中引入一定程度的随机性。为了实现这一点，我们将修改模型响应的**温度** 。

## 采样和温度
｜Sampling and Temperature

在语言模型领域，**采样（sampling）** 是指模型通过对潜在下一个单词（实际上是标记（tokens），但出于我们的目的，我们可以将标记视为单词）的概率分布进行采样来生成文本的过程。当我们在没有启用**采样** 的情况下与语言模型交互时，模型以确定性方式运行，在生成文本时始终选择最可能的下一个单词。当您需要一致性和准确性时，这种默认行为很有用，但当您追求创造性和多样化的响应时，它可能会受到限制。


在用于与我们的 LLaMA-2 模型交互的`transformers`管道上下文中，默认情况下**采样** 是*禁用的*。要启用采样，我们在调用我们的`generate`函数时设置`do_sample=True`，并在这样做时，指示模型根据概率分布选择单词，允许选择可能性较低的单词，这可能导致更具多样性和趣味性的文本。


启用**采样** 后，我们还可以为**温度（temperature）** 传递特定的值，您可以将其视为响应的随机程度。对于`temperature`，我们传递一个介于`0.0`和`1.0`之间的值，较大的值表示较大的随机程度。


## 练习：随机响应
｜Exercise: Random Responses

重用与上面相同的**系统上下文** 和提示，启用**采样** （`do_sample=True`）并将**温度** 设置为其最高值（`temperature=1.0`）。生成 3 个不同的响应以检查它们是否都唯一。

如果您遇到困难，请查看下面的解决方案。

## 您的工作在此处

## 解决方案

In [ ]:
print(generate(construct_prompt_with_context(prompt, system_context), do_sample=True, temperature=1))

In [ ]:
print(generate(construct_prompt_with_context(prompt, system_context), do_sample=True, temperature=1))

In [ ]:
print(generate(construct_prompt_with_context(prompt, system_context), do_sample=True, temperature=1))

## 创造力和准确性
｜Creativity and Accuracy

作为对`temperature`的最后思考，值得一提的是，随机生成虽然在独特或创造性输出的上下文中很有用，但与响应的精确性和准确性并不特别一致。在需要模型生成精确和/或事实准确响应的上下文中，在提高温度时始终注意检查其输出。

## 关键概念回顾

本笔记本中介绍了以下关键概念：

- **采样（Sampling）：** 文本生成（text generation）中的一种过程，其中语言模型（language model）根据词汇表上的概率分布（probability distribution）选择下一个单词（token）。
- **温度（Temperature）：** 一个超参数（hyperparameter），它控制模型在采样过程（sampling）中预测的随机程度（level of randomness）。较高的温度（temperature）会增加多样性（diversity），从而导致更具变化性和创造性的响应（varied and creative responses），而较低的温度会使模型的输出更具可预测性（predictable）和保守性（conservative）。

## 可选的高级练习

如果您想超越课程的要求，以下是一些供您尝试的额外开放式练习。

### 使用 7B 模型

在笔记本的顶部，在重新启动内核（请参见下面的单元格）之后，取消注释并使用 7B 模型而不是我们演示的 13B 模型。尝试在使用较小（较弱）模型的情况下获得令人满意的结果。

### 创建交互式角色

现在您能够从虚构人物那里生成陈述，请尝试创建一个小型系统，该系统创建多个不同的个性并让他们相互交互。

### 让交互式角色玩游戏

扩展之前的练习，创建多个角色，他们朝着您创建的某个“游戏”定义的目标努力。这可能是试图让另一个角色说出某个特定的词语，泄露他们隐藏的宝藏的秘密位置，甚至是一些合作性的东西，角色必须共同努力才能实现某个目标。如果您真的想挑战一下，您甚至可以考虑创建两个以上的角色，甚至创建相互交互的玩家团队。

## 重新启动内核
｜Restart the Kernel

为了释放下一个笔记本的 GPU 内存，请运行以下单元格以重新启动内核。

In [ ]:
from IPython import get_ipython

get_ipython().kernel.do_shutdown(restart=True)

# [5LOI DEEP LEARNING INSTITUE](https://5loi.com)

### [AI COMMUNITY](https://www.theforage.cn/community)

### [5LOI](https://5loi.com/about_loi)

![DLI Header](images/DLI_Header.png)